# **Azure AI Agents with Knowledge Tools (Bing Search)**

## Overview
This notebook demonstrates how to create and use AI agents with knowledge tools in Azure AI Foundry. You'll learn how to enhance an AI agent's capabilities by integrating external knowledge sources like Bing search, allowing it to provide up-to-date information in response to user queries.

## Understanding Stateless vs. Stateful APIs

### **Completions API (Stateless)**
In the previous notebooks, we used the OpenAI Completions API, which is **stateless**:
- Each request is independent and does not retain memory of previous interactions.
- Conversation history must be manually included in every request.
- Simple request-response pattern with no built-in conversation management.

```python
# Stateless Completions API example
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me about Mars."}
    ]
)
```

#### **Agent API (Stateful) workflow**
- Maintains conversation context through threads and runs.
- Automatically manages conversation history and state.
- Enables persistence of interactions for multi-turn conversations.

**1. Create a thread (conversation container)**
```python
thread = project_client.agents.create_thread()
```

**2. Add messages to the thread**
```python
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Tell me about Mars."
)
```

**3. Run the agent on the thread**
```python
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, 
    agent_id=agent.id
)

## 1. Setting Up the Environment

First, we'll import the necessary libraries and load environment variables from a `.env` file.
This provides access to connection strings, model names, and other configuration details.

In [1]:
import os
import dotenv
dotenv.load_dotenv(".env")

True

### Creating the Azure AI Project Client

Now we'll establish a connection to our Azure AI Foundry project using the connection string from our environment variables. This client will be used to manage agents, threads, and knowledge tools.

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

## 2. Setting Up Knowledge Tools

Knowledge tools extend an agent's capabilities by providing access to external information sources. Here, we'll create a Bing search tool that allows our agent to search the web for current information.

The `BingGroundingTool` integrates with Bing Search to enable:
- Real-time information retrieval
- Access to current events and news
- Factual grounding for agent responses

In [3]:
from azure.ai.projects.models import BingGroundingTool

bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"]
)
conn_id = bing_connection.id

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

## 3. Defining Agent Instructions

System instructions guide how the agent behaves and uses its tools. Here we create detailed instructions that specify:
- When and how to use Bing search
- How to cite information sources properly using IEEE format
- Guidelines for summarizing and presenting information
- How to handle limitations in search results

These instructions serve as the agent's "operating manual" for processing queries.

In [4]:
system_message=(
    "You are a knowledgeable assistant with access to Bing search capabilities."
    "\n When answering questions:"
    "\n     1. For factual queries, use the Bing tool to search for up-to-date information."
    "\n     2. Always cite your sources when providing information from Bing."
    "\n     3. If the search results are insufficient, acknowledge limitations and offer what you know."
    "\n     4. Summarize complex information in a clear, concise manner."
    "\n     5. When appropriate, organize information with bullet points or numbered lists."
    "\n     6. Avoid making claims that aren't supported by your search results."
    "\n     7. For time-sensitive information, note when the data was retrieved."
    "\n     8. If a query is ambiguous, ask clarifying questions before searching."
    "\n     9. If the user asks for a specific format (e.g., table), provide it if possible."
    "\n     10. For citations, please use the IEEE (Institute of Electrical and Electronics Engineers) format."
    "\n           - How you should apply it:"
    "\n                 a. For in-text citations, use numbered citations in brackets [1]."
    "\n                 b. At the end of the report, you should provide a list of citations in the format "
    "(the list should ONLY contain the sources used in the free text of the research report. "
    "Do NOT list sources which are not cited in the free text of the research report.):"
    "\n                     [1] Title of the source, URL."
    "\n                 c. The list should be numbered in the order they appear in the text."
    "\n                 d. If the same source/url is cited multiple times, it should be listed only once in the citation list."
    "\n                 e. Pay extra attention in the end so that you do not create multiple citations lists."
    "\n\nYour goal is to provide accurate, helpful responses while clearly distinguishing between search results and your own knowledge."
)

# print(system_message)

## 4. Creating the Agent

Now we'll create the actual agent with our system instructions and Bing tool. The agent combines:
- A language model (specified in our environment variables)
- Our detailed system instructions
- Access to the Bing search tool

This creates a capable assistant that can search the web for information when responding to queries.

In [5]:
from azure.ai.projects.models import FunctionTool

agent = project_client.agents.create_agent(
    name="my-knowledge-tool-agent",
    model=os.getenv("chatModel"),
    instructions=system_message,
    tools=bing.definitions
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_tEXMIS5k4qByweU3u03KWDE7


## 5. Creating a Thread

Threads are conversation containers in the Agent API. Each thread:
- Holds the history of messages between user and agent
- Maintains conversation state and context
- Enables multi-turn conversations

Think of a thread as a dedicated, persistent chat session for a specific conversation.

In [ ]:
thread = project_client.agents.create_thread()

## 6. Adding a Message to the Thread

Now we'll add our first user message to the thread. This message will be processed by the agent when we run it.
We're asking about the latest news related to OpenAI, which will require the agent to use its Bing search tool to find current information.

In [7]:
from azure.ai.projects.models import MessageRole

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="What is the latest news in AI when it comes to OpenAI?",
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_LfhQEIVy8Kk8bf9Yg8OKxpzS


## 7. Running the Agent

Now we'll run the agent on our thread. During this process, the agent will:
1. Read the message asking about OpenAI news
2. Determine that real-time information is needed
3. Use the Bing search tool to find current news articles
4. Synthesize the information into a comprehensive response
5. Format the response with proper citations

The `create_and_process_run` method handles all of these steps in a single call.

In [8]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: completed


### Viewing Thread Messages

Now let's examine the conversation thread to see the agent's response. The `pretty_print_thread_messages` helper function formats the conversation in a readable way, showing both user queries and agent responses.

In [9]:
from utils.helper import pretty_print_thread_messages

messages = project_client.agents.list_messages(thread_id=thread.id)
pretty_print_thread_messages(messages)


👤 USER
------------------------------------------------------------


What is the latest news in AI when it comes to OpenAI?


🤖 ASSISTANT
------------------------------------------------------------


OpenAI recently paused free access to its GPT-4o image-generation feature after a viral trend where users created  
Studio Ghibli-style images, raising legal and ethical concerns. While paid subscribers retain access, OpenAI is    
reevaluating due to copyright implications and the overwhelming popularity of the feature[3][2][1].


Citations:
1. The OpenAI Studio Ghibli controversy could be a test for art copyright ...: https://tech.yahoo.com/articles/openai-studio-ghibli-controversy-could-060000378.html
2. OpenAI's GPT-4o Generates Studio Ghibli-Style Images Amid Copyright ...: https://www.binance.com/en/square/post/03-28-2025-openai-s-gpt-4o-generates-studio-ghibli-style-images-amid-copyright-concerns-22134493396594
3. OpenAI pauses free GPT-4o image generation after viral Studio Ghibli ...: https://www.techedt.com/openai-pauses-free-gpt-4o-image-generation-after-viral-studio-ghibli-trend


## 8. Continuing the Conversation

One of the advantages of the Agent API is maintaining conversation context. Let's add a follow-up question that references our first query. The agent will understand that "Which industries" refers to industries affected by OpenAI's recent developments mentioned in the previous exchange.

In [10]:
from azure.ai.projects.models import MessageRole

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Which industries are most affected?",
)

### Processing the Follow-up Question

Now we'll run the agent again to process our follow-up question. The agent will:
1. Understand the context from the previous exchange
2. Use Bing search to find information about industries affected by OpenAI's developments
3. Generate a contextually relevant response that builds on the previous information

In [11]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: completed


### Viewing the Complete Conversation

Let's see the full conversation thread, which now includes our follow-up question and the agent's response. This demonstrates how the agent maintains context across multiple turns of a conversation.

In [12]:
from utils.helper import pretty_print_thread_messages

messages = project_client.agents.list_messages(thread_id=thread.id)
pretty_print_thread_messages(messages)


👤 USER
------------------------------------------------------------


What is the latest news in AI when it comes to OpenAI?


🤖 ASSISTANT
------------------------------------------------------------


OpenAI recently paused free access to its GPT-4o image-generation feature after a viral trend where users created  
Studio Ghibli-style images, raising legal and ethical concerns. While paid subscribers retain access, OpenAI is    
reevaluating due to copyright implications and the overwhelming popularity of the feature[3][2][1].


Citations:
1. The OpenAI Studio Ghibli controversy could be a test for art copyright ...: https://tech.yahoo.com/articles/openai-studio-ghibli-controversy-could-060000378.html
2. OpenAI's GPT-4o Generates Studio Ghibli-Style Images Amid Copyright ...: https://www.binance.com/en/square/post/03-28-2025-openai-s-gpt-4o-generates-studio-ghibli-style-images-amid-copyright-concerns-22134493396594
3. OpenAI pauses free GPT-4o image generation after viral Studio Ghibli ...: https://www.techedt.com/openai-pauses-free-gpt-4o-image-generation-after-viral-studio-ghibli-trend

👤 USER
------------------------------------------------------------


Which industries are most affected?


🤖 ASSISTANT
------------------------------------------------------------


The industries most affected by AI developments like OpenAI's innovations, including its advanced capabilities such
as image generation and language models, include:                                                                  

 1 Creative Industries:                                                                                            
    • Media and Entertainment: AI-generated art styles (e.g., Studio Ghibli trend mentioned above) enable creators 
      to explore new aesthetics but also pose copyright risks.                                                     
    • Gaming: AI generates customized graphics and narratives.                                                     
    • Publishing: Automated content creation for blogs, articles, or creative writing.                             
 2 Healthcare:                                                                                                     
    • AI assists in diagnostics (e.g., medical imaging) and treatment recommendations, improving patient care      
      efficiency.                                                                                                  
    • Integration of AI chatbots for mental health and patient consultation.                                       
 3 Education and Training:                                                                                         
    • Personalized learning experiences through AI tutoring systems.                                               
    • Creation of educational content using AI-based tools.                                                        
 4 Advertising and Marketing:                                                                                      
    • AI optimizes campaigns using data-driven strategies and generates vivid marketing visuals.                   
    • Tools developed by OpenAI help automate copywriting and content brainstorming.                               
 5 Legal and Regulatory:                                                                                           
    • Legal sectors increasingly adopt AI for document review and drafting contracts.                              
    • At the same time, copyright and ethical concerns stemming from AI-generated content include significant      
      policy impacts.                                                                                              

These advancements have brought immense possibilities but also ongoing discussions about ethical and legal         
regulations, especially around intellectual property. OpenAI's recent halt of free access to GPT-4o                
image-generation highlights such concerns as AI reshapes creative processes[3][2][1].


Citations:
1. The OpenAI Studio Ghibli controversy could be a test for art copyright ...: https://tech.yahoo.com/articles/openai-studio-ghibli-controversy-could-060000378.html
2. OpenAI's GPT-4o Generates Studio Ghibli-Style Images Amid Copyright ...: https://www.binance.com/en/square/post/03-28-2025-openai-s-gpt-4o-generates-studio-ghibli-style-images-amid-copyright-concerns-22134493396594
3. OpenAI pauses free GPT-4o image generation after viral Studio Ghibli ...: https://www.techedt.com/openai-pauses-free-gpt-4o-image-generation-after-viral-studio-ghibli-trend


## 9. Cleanup

When we're finished with our agent and thread, it's good practice to clean up these resources. This helps manage resource usage and keeps your environment tidy.

In a production environment, you might retain threads for longer periods to maintain conversation history, but for this demonstration we'll delete both the agent and thread.

In [13]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

# Delete Thread when done
project_client.agents.delete_thread(thread_id=thread.id)
print(f"thread: {thread.id} deleted")

Deleted agent
thread: thread_wh17E6I63RgGUFvtoQhigF1V deleted
